In [ ]:
import cv2
import numpy as np
import pywt
import time
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def enhance_image_clahe(image):
    yuv_img = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    yuv_img[:, :, 0] = clahe.apply(yuv_img[:, :, 0])
    return cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)

def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def enhance_image_gamma_clahe(image):
    return enhance_image_clahe(gamma_correction(image, gamma=1.5))

def wavelet_denoise(image, wavelet='haar', level=1):
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    coeffs_H = list(coeffs)
    coeffs_H[1:] = [tuple([np.zeros_like(v) for v in detail]) for detail in coeffs_H[1:]]
    return np.uint8(pywt.waverec2(coeffs_H, wavelet))

def enhance_image_wavelet_clahe(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised_image_gray = wavelet_denoise(image_gray)
    denoised_image_bgr = cv2.cvtColor(denoised_image_gray, cv2.COLOR_GRAY2BGR)
    return enhance_image_clahe(denoised_image_bgr)

def add_noise(image, noise_factor=0.2):
    noise = np.random.normal(loc=0, scale=noise_factor, size=image.shape)
    noisy_image = np.clip(image / 255.0 + noise, 0, 1) * 255.0
    return noisy_image.astype(np.uint8)

def msrcr(image, sigma_list=[15, 80, 250]):
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    img_float = image.astype(np.float32) / 255.0
    
    # Convert to log domain
    img_log = np.log1p(img_float)
    
    # Multi-scale
    retinex = np.zeros_like(img_log)
    for sigma in sigma_list:
        gaussian = gaussian_filter(img_log, sigma=sigma)
        retinex += (img_log - gaussian)
    
    retinex = retinex / len(sigma_list)
    
    # Color restoration
    img_sum = np.sum(img_float, axis=2, keepdims=True)
    color_restoration = np.log1p(img_float * 125.0 / (img_sum + 1e-6))
    
    # Combine
    msrcr = retinex * color_restoration
    
    # Normalize and convert back to uint8
    msrcr = (msrcr - np.min(msrcr)) / (np.max(msrcr) - np.min(msrcr))
    msrcr = (msrcr * 255).astype(np.uint8)
    
    return msrcr

import cv2
import numpy as np
import pywt
import time
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, median_filter
import pandas as pd

# Existing functions (enhance_image_clahe, gamma_correction, etc.) remain the same

def spatial_filtering(image):
    # Apply Gaussian and median filters
    gaussian_filtered = gaussian_filter(image, sigma=1)
    median_filtered = median_filter(image, size=3)
    return np.uint8(gaussian_filtered), np.uint8(median_filtered)

def evaluate_efficiency(original_image):
    # Record enhancement times and quality metrics for various methods
    methods = {
        "CLAHE": enhance_image_clahe,
        "Gamma+CLAHE": enhance_image_gamma_clahe,
        "Wavelet+CLAHE": enhance_image_wavelet_clahe,
        "MSRCR": msrcr
    }
    
    # Initialize results list
    results = []

    for method_name, method in methods.items():
        start_time = time.time()
        enhanced_image = method(original_image)
        exec_time = time.time() - start_time
        
        # Convert images to grayscale
        image_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        enhanced_gray = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)

        # Calculate PSNR and SSIM
        psnr_value = psnr(image_gray, enhanced_gray)
        ssim_value = ssim(image_gray, enhanced_gray)

        # Append results
        results.append([method_name, exec_time, psnr_value, ssim_value])
    
    # Apply spatial filtering techniques
    gaussian_filtered, median_filtered = spatial_filtering(cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY))
    
    # Calculate PSNR and SSIM for Gaussian and Median filters
    psnr_gaussian = psnr(image_gray, gaussian_filtered)
    ssim_gaussian = ssim(image_gray, gaussian_filtered)
    psnr_median = psnr(image_gray, median_filtered)
    ssim_median = ssim(image_gray, median_filtered)
    
    # Append results for spatial filters
    results.append(["Gaussian Filter", 0, psnr_gaussian, ssim_gaussian])
    results.append(["Median Filter", 0, psnr_median, ssim_median])

    # Convert results to DataFrame for table format
    results_df = pd.DataFrame(results, columns=["Method", "Time (s)", "PSNR", "SSIM"])
    print(results_df)

    # Plotting images
    plt.figure(figsize=(15, 8))
    plt.subplot(2, 4, 1); plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)); plt.title("Original")
    for i, (method_name, method) in enumerate(methods.items(), start=2):
        plt.subplot(2, 4, i)
        enhanced_image = method(original_image)
        plt.imshow(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))
        plt.title(method_name)
    plt.subplot(2, 4, 6); plt.imshow(gaussian_filtered, cmap='gray'); plt.title("Gaussian Filter")
    plt.subplot(2, 4, 7); plt.imshow(median_filtered, cmap='gray'); plt.title("Median Filter")
    plt.tight_layout(); plt.show()

def process_multiple_images(image_paths):
    for image_path in image_paths:
        original_image = cv2.imread(image_path)
        if original_image is not None:
            print(f"Processing: {image_path}")
            noisy_image = add_noise(original_image)
            evaluate_efficiency(noisy_image)
        else:
            print(f"Error loading {image_path}")

# Example usage
image_paths = [
    r"C:\Amrita\Sem-5\Digital Image Processing\images\00000010_000.png"
    # Add more image paths as needed
]
process_multiple_images(image_paths)